In [1]:
!nvidia-smi

Thu Dec 22 21:13:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:03:00.0 Off |                  Off |
| N/A   45C    P0    39W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tokenizers

In [3]:
!pip install transformers tinydb

In [4]:
!pip install pytorch_lightning==1.2.10 

In [5]:
!pip install torchtext==0.10.0

ERROR: Could not find a version that satisfies the requirement torchtext==0.10.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1)
ERROR: No matching distribution found for torchtext==0.10.0


In [6]:
!gdown https://drive.google.com/uc?id=1AGtoZDyHNm8FyGqwrIFijAE4_CarvwrT
!gdown https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4

Downloading...
From: https://drive.google.com/uc?id=1AGtoZDyHNm8FyGqwrIFijAE4_CarvwrT
To: /home/user/pictogram_prediction_pt/pictogram_sentences_final_final.json
100%|██████████████████████████████████████| 8.27M/8.27M [00:00<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n9LRtmKx-uTTIkCWHLgeTpPB_-bp7cn4
To: /home/user/pictogram_prediction_pt/pictograms.json
100%|██████████████████████████████████████| 7.78M/7.78M [00:00<00:00, 48.0MB/s]


In [7]:
import json
corpus = json.load(open("./pictogram_sentences_final_final.json","r"))

In [8]:
id_sentences = []
ids_ = []
for sentence in corpus:
  s = []
  for t in sentence['pictograms']:
    if t["id"] is None:
      s.append(t["caption"])
      ids_.append(t["caption"])
    else:
      ids_.append(str(t["id"]))
      s.append(str(t["id"]))
  if len(s) >= 3:
    id_sentences.append(" ".join(s))

In [9]:
ids_ = list(set(ids_))
len(ids_)

3675

In [10]:
CORPUS_PATH = "./corpus.txt"
f = open(CORPUS_PATH,"w")
for s in id_sentences:
  f.write(s+"\n")
f.close()

## Criar tokenizer

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.processors import BertProcessing

special_tokens = ["[UNK]","[SEP]", "[CLS]", "[PAD]", "[MASK]"]

sense_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
sense_tokenizer.add_special_tokens(special_tokens)
sense_tokenizer.pre_tokenizer = WhitespaceSplit()

sep_token = "[SEP]"
cls_token = "[CLS]"
pad_token = "[PAD]"
unk_token = "[UNK]"
sep_token_id = sense_tokenizer.token_to_id(str(sep_token))
cls_token_id = sense_tokenizer.token_to_id(str(cls_token))
pad_token_id = sense_tokenizer.token_to_id(str(pad_token))
unk_token_id = sense_tokenizer.token_to_id(str(unk_token))

sense_tokenizer.post_processor = BertProcessing(
                (str(sep_token), sep_token_id), (str(cls_token), cls_token_id)
            )

In [12]:
from tokenizers.trainers import WordLevelTrainer
g = WordLevelTrainer(vocab_size = len(ids_)+5,special_tokens=special_tokens,show_progress=True)
sense_tokenizer.train(files=[CORPUS_PATH],trainer=g)

In [13]:
TOKENIZER_PATH = "tokenizer.json"
sense_tokenizer.save(TOKENIZER_PATH)

In [14]:
tokenizer_tokens = set(list(sense_tokenizer.get_vocab().keys()))
len(tokenizer_tokens)

3668

## Ajustar a matriz de embeddings do BERTimbau.

In [15]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertForMaskedLM
BASE_MODEL = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = BertForMaskedLM.from_pretrained(BASE_MODEL)
finetune_model = BertForMaskedLM.from_pretrained(BASE_MODEL)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializi

In [16]:
from transformers import PreTrainedTokenizerFast

cs_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
cs_tokenizer.pad_token = "[PAD]"
cs_tokenizer.sep_token = "[SEP]"
cs_tokenizer.mask_token = "[MASK]"
cs_tokenizer.cls_token = "[CLS]"
cs_tokenizer.unk_token = "[UNK]"

In [17]:
from torch import tensor
# Get weights of the old wte
old_wgts = finetune_model.get_input_embeddings().weight.clone().detach()
print(old_wgts.size())
# Get the mean embedding vetor of the old wte
wgts_m = old_wgts.mean(0)

# Initialize vocab size and weights of the new wte
new_vocab_size = len(cs_tokenizer.vocab)
bert_embeddings = model.get_input_embeddings()

new_wgts = old_wgts.new_zeros(new_vocab_size,old_wgts.size(1))
new_wgts.size()

vectors_m = old_wgts.mean(0)

torch.Size([29794, 768])


In [18]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import requests

model_name_or_path = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
vit_model = ViTModel.from_pretrained(model_name_or_path)
vit_model = vit_model.to("cuda:0")


Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import torch
def get_pictogram_embeddings(id_):
  url = "https://api.arasaac.org/api/pictograms/{0}?download=false".format(id_)
  
  try:
    image = Image.open(requests.get(url, stream=True).raw)
    # image = Image.open("aqui.jpg")
  except:
    print('n')
    return None
  image = image.convert('RGB')


  inputs = feature_extractor(images=image, return_tensors="pt")
  inputs = inputs.to("cuda:0")
  outputs = vit_model(**inputs)
  del inputs
  torch.cuda.empty_cache()
  sequence_output = outputs[0].detach().cpu()
  return sequence_output[0, 0, :]
  
embs = get_pictogram_embeddings("21945")
# embs

/home/user/miniconda3/envs/fastai/lib/python3.10/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [20]:
from tinydb import TinyDB,Query
import re, torch
Pictogram = Query()
Keyword = Query()
db = TinyDB("pictograms.json")
picograms = db.table("pictogram")

def get_by_id(id_):
  pics = picograms.search(Pictogram._id == int(id_))
  if len(pics) > 0:
    return pics[0]
  return None

def get_related_embeddings(w):
  tokens = []
  if w.isnumeric():
    pictogram = get_by_id(w)
    if pictogram is not None:
      tokens = [k['keyword'] for k in pictogram['keywords']]
    else:
      tokens = [w]  
  else:
    tokens = [w]
  
  ws = " ".join(tokens)
  ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ws))
  v_ = bert_embeddings(torch.tensor(ids)).mean(0)
    
  return v_

# get_related_embeddings("21945")

In [21]:
import torch 
from tqdm import tqdm

for w,i in list(cs_tokenizer.get_vocab().items()):
  
  tokens = []
  related_e = get_related_embeddings(w)
  if w.isnumeric():
    # print(w)
    pictogram = get_pictogram_embeddings(w)
    if pictogram is not None:
      new_wgts[i] = pictogram+related_e
      continue
    else:
      tokens = [w]  
  else:
    tokens = [w]
  
  if len(tokens) > 0:
    ws = " ".join(tokens)
    # print("ww",ws)
    ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ws))
    v_ = bert_embeddings(torch.tensor(ids)).mean(0)
    new_wgts[i] = v_

n
n
n
n
n
n
n
n


In [22]:
from torch import nn

new_wte = nn.Embedding(new_vocab_size,old_wgts.size(1)) # new embeddings
new_wte.weight.data.normal_(mean=0,std=finetune_model.config.initializer_range) 
new_wte.weight.data = new_wgts

finetune_model.resize_token_embeddings(len(cs_tokenizer))
finetune_model.set_input_embeddings(new_wte)
finetune_model.set_output_embeddings(new_wte)

In [23]:
# finetune_model.save_pretrained('/content/gdrive/MyDrive/Doutorado/Corpus construction/execution/files/adjusted-bertimbau-base-14jul')
BASE_MODEL = './image_related_model'
finetune_model.save_pretrained(BASE_MODEL)

## Treinamento

In [24]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [25]:
from transformers import PreTrainedTokenizerFast
TOKENIZER_PATH = "tokenizer.json"
cs_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
cs_tokenizer.pad_token = "[PAD]"
cs_tokenizer.sep_token = "[SEP]"
cs_tokenizer.mask_token = "[MASK]"
cs_tokenizer.cls_token = "[CLS]"
cs_tokenizer.unk_token = "[UNK]"

In [26]:
f = open("corpus.txt","r")
id_sentences = f.readlines()
id_sentences = [l.rstrip() for l in id_sentences]
id_sentences[0]

'31670 36392 31408 7074 7031 próprio 3218'

In [51]:
# BASE_MODEL = './modelo'
BATCH_SIZE = 768
NUM_WORKERS = 6
MAX_EPOCHS = 500
WARMUP_STEPS = int(MAX_EPOCHS * 0.1)

LEARNING_RATE = 0.00001

LOGS_PATH = "img_logs_500"
LOGGER_INFO = "500"
LOGGER_VERSION = "1e5"
CHECKPOINTS_PATH = "img_checkpoints_500"
GPUS = 1
ACCUMULATE_GRAD_BATCHES = 4

In [52]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.33

train_idx, val_idx = train_test_split(list(range(len(id_sentences))), test_size=TEST_SIZE, random_state=32)
test_idx, val_idx = train_test_split(val_idx, test_size=0.5, random_state=3)

In [53]:
import numpy as np
train_examples = np.array(id_sentences).take(train_idx)
val_examples = np.array(id_sentences).take(val_idx)
test_examples = np.array(id_sentences).take(test_idx)
# test_examples = human_sentences

len(train_examples), len(val_examples),len(test_examples)

(8329, 2052, 2051)

In [54]:
max_len = 13

def tokenize_function(tokenizer,examples):
    # Remove empty lines
    examples = [line for line in examples if len(line) > 0 and not line.isspace()]
    bert = tokenizer(
        examples,
        padding="max_length",
        max_length=max_len,
        return_special_tokens_mask=True,
        truncation=True
    )
    return bert

val_tokenized_examples = tokenize_function(cs_tokenizer,val_examples)
test_tokenized_examples = tokenize_function(cs_tokenizer,test_examples)
train_tokenized_examples = tokenize_function(cs_tokenizer,train_examples)

In [55]:
from torch.utils.data import Dataset, Subset
from torch import tensor

class MyDataset(Dataset):
  def __init__(self, examples):

    self.input_ids = examples['input_ids']
    self.attention_mask = examples['attention_mask']
    self.special_tokens_mask = examples['special_tokens_mask']
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    input_ids = tensor(self.input_ids[idx])
    attention_mask = tensor(self.attention_mask[idx])
    special_tokens_mask = tensor(self.special_tokens_mask[idx])

    return {
      "input_ids":input_ids, 
      "attention_mask":attention_mask, 
      "special_tokens_mask":special_tokens_mask
    }

train_dataset = MyDataset(train_tokenized_examples)
test_dataset = MyDataset(test_tokenized_examples)
val_dataset = MyDataset(val_tokenized_examples)


In [56]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=cs_tokenizer, mlm_probability=0.15)
data_collator2 = DataCollatorForLanguageModeling(tokenizer=cs_tokenizer,mlm=False)

In [57]:
def top_n_data_collator(examples):
  
  batch = {
      "input_ids" : torch.stack([example['input_ids'] for example in examples]),
      "attention_mask": torch.stack([example['attention_mask'] for example in examples]),
  }
  special_tokens_mask = torch.stack([example['special_tokens_mask'] for example in examples])
  masked_indices = special_tokens_mask.bool()

  mask_labels = special_tokens_mask.clone()
  probability_matrix = mask_labels
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
  masked_indices = probability_matrix.bool()

  labels = batch['input_ids'].clone()
  
  for i, row in enumerate(special_tokens_mask.bool()):
    # print(torch.where(row==False)[0])
    mask_id = torch.where(row==False)[0][-2]
    masked_indices[i][mask_id] = True
    batch['input_ids'][i][mask_id] = cs_tokenizer.mask_token_id


  labels[~masked_indices] =  -100
  
  batch['labels'] = labels

  return batch

# top_n_data_collator(train_dataset)

In [58]:
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    # drop_last = True,
    shuffle=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    # drop_last = True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=data_collator2,
    pin_memory=True,
)

test_dataloader_topn = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=top_n_data_collator,
    pin_memory=True,
)

In [59]:
from argparse import ArgumentParser
import math
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import get_polynomial_decay_schedule_with_warmup
from transformers import AdamW
import torch.nn.functional as F

# BertForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")

from transformers import BertLMHeadModel
from transformers import BertForMaskedLM


class LitBertClassifier(pl.LightningModule):
    def __init__(self, pretrained_model_name='bert-large-uncased'):
        super().__init__()
        self.save_hyperparameters()
        self.batch_size = BATCH_SIZE
        self.lr = LEARNING_RATE
        self.train_dataset = train_dataset
        self.bert = BertForMaskedLM.from_pretrained(pretrained_model_name)
        self.topns =[1,9,18,25,36]
      
    
    def freeze_to(self, layers):
      for param in self.bert.bert.encoder.layer[:layers].parameters():
        param.requires_grad = False


    def forward(self, input_ids, attention_mask, labels=None):
        if labels == None:
            return self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )    
        return self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels = labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self._shared_step(batch, batch_idx)
        loss = outputs[0]

        self.log("train_loss", loss, on_epoch=True, prog_bar=True,)

        return loss

    # def training_epoch_end(self, outputs):
    #   #print(outputs)
    #   train_loss = torch.stack([x['loss'] for x in outputs]).mean()
    #   self.log("train_loss", train_loss, on_epoch=True, prog_bar=True,)

    def train_dataloader(self):
      return DataLoader(self.train_dataset,batch_size=self.batch_size,num_workers=NUM_WORKERS,pin_memory=True,collate_fn=data_collator)
      # return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def get_accuracy(self, batch, results):
        y_true = tensor([]).to(torch.device("cuda:0"))
        y_pred = tensor([]).to(torch.device("cuda:0"))
        for idx, ipids in enumerate(batch["input_ids"]):

            idxs = (ipids == tokenizer.mask_token_id).nonzero()
            if idxs.size()[0] > 0:
                device = "cuda:0"
                y_true = torch.cat((y_true, batch['labels'][idx][idxs].resize(1, idxs.size()[0])[0]))
                # y_true = y_true + batch['labels'][idx][idxs].resize(1, idxs.size()[0])[0].tolist()

                idxs_2 = tensor([a[0] for a in idxs])
                idxs_2 = idxs_2.to(device)
                res = torch.index_select(results[idx],0,idxs_2).argmax(1)

                y_pred = torch.cat((y_pred,res))
        if len(y_pred) == 0:
            return None
        return accuracy(tensor(y_true),tensor(y_pred))

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()
          # return {"loss":loss}
          # perplexity = torch.exp(loss)
          # acc = self.get_accuracy(batch,result[1].clone())
          # self.log("val_ppl", perplexity, on_epoch=True, prog_bar=True,)
          # self.log("val_acc", acc, on_epoch=True, prog_bar=True,)
          # self.log("val_loss", loss, on_epoch=True, prog_bar=True,)

          return {
              "val_loss":loss
          }
    
    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log("val_loss", val_loss, on_epoch=True, prog_bar=True,)
    
    
    def test_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()
          perplexity = torch.exp(loss)
          self.log("test_ppl", perplexity, on_epoch=True, prog_bar=True,)
          self.log("test_loss", loss, on_epoch=True, prog_bar=True,)

          if cs_tokenizer.mask_token_id in batch["input_ids"][0]:

            predictions = F.softmax(result[1], dim=-1)
            labels = batch['labels']
            masked = batch['input_ids']
            n = masked.detach().cpu().numpy()
            predicted = predictions.detach().cpu().numpy()[n == cs_tokenizer.mask_token_id]

            ids = np.argsort(-1*predicted,axis=1)
            for first in self.topns:
              count = 0
              for i, data in enumerate(ids):
                if labels[masked == cs_tokenizer.mask_token_id][i].item() in data[:first]:
                  count += 1
              isin = count/predicted.shape[0]

              self.log("test_"+str(first), isin, on_epoch=True, prog_bar=True,)

          return {
              "test_ppl":perplexity,
              "test_loss":loss
          }
    

    def _shared_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = self.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        return outputs

    def configure_optimizers(self):
      # optimizer =  AdamWeightDecayOptimizer(self.parameters(), lr=LEARNING_RATE,betas=(0.9, 0.999), eps=1e-6)
      # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=1735, epochs=500,anneal_strategy='linear')
      # return [optimizer],[scheduler]
      optimizer = AdamW(self.parameters(), lr=self.lr,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
      scheduler = {
          'scheduler': get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,lr_end=1e-08),
          # 'scheduler': get_linear_schedule_with_warmup(optimizer,num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,last_epoch=-1),
          'name': 'lr'
      }
      return [optimizer],[scheduler]
    
    def backward(self, loss, optimizer, idx):
        loss.backward()
    


In [60]:
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor

tb_logger = pl_loggers.TensorBoardLogger(LOGS_PATH,name=LOGGER_INFO, version=LOGGER_VERSION)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [61]:

checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINTS_PATH,
    filename='bertimbau-base-{epoch:02d}-{val_loss:.2f}',
    mode='min',
    save_last=True
)

In [62]:
trainer = pl.Trainer(
    accelerator='ddp',
    max_epochs=MAX_EPOCHS,
    logger=tb_logger,
    gpus=GPUS,
    callbacks=[checkpoint_callback,lr_monitor],
    precision=16,
    auto_scale_batch_size="binsearch"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


In [63]:
bert = LitBertClassifier(BASE_MODEL)

In [64]:
# trainer.tune(bert)

In [65]:
trainer.fit(bert,train_dataloader,val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 88.9 M
-----------------------------------------
88.9 M    Trainable params
0         Non-trainable params
88.9 M    Total params
355.454   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving latest checkpoint...


1

In [66]:
trainer.test(bert,test_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 4.806456565856934, 'test_ppl': 122.40672302246094}
--------------------------------------------------------------------------------


[{'test_ppl': 122.40672302246094, 'test_loss': 4.806456565856934}]

In [67]:
trainer.test(bert,test_dataloaders=test_dataloader_topn)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1': 0.04193076491355896,
 'test_18': 0.22038030624389648,
 'test_25': 0.2549975514411926,
 'test_36': 0.29254022240638733,
 'test_9': 0.16918575763702393,
 'test_loss': 7.166921138763428,
 'test_ppl': 1296.712890625}
--------------------------------------------------------------------------------


[{'test_ppl': 1296.712890625,
  'test_loss': 7.166921138763428,
  'test_1': 0.04193076491355896,
  'test_9': 0.16918575763702393,
  'test_18': 0.22038030624389648,
  'test_25': 0.2549975514411926,
  'test_36': 0.29254022240638733}]

In [68]:
trainer.test(bert,test_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1': 0.04193076491355896,
 'test_18': 0.22038030624389648,
 'test_25': 0.2549975514411926,
 'test_36': 0.29254022240638733,
 'test_9': 0.16918575763702393,
 'test_loss': 4.806456565856934,
 'test_ppl': 122.40672302246094}
--------------------------------------------------------------------------------


[{'test_ppl': 122.40672302246094,
  'test_loss': 4.806456565856934,
  'test_1': 0.04193076491355896,
  'test_9': 0.16918575763702393,
  'test_18': 0.22038030624389648,
  'test_25': 0.2549975514411926,
  'test_36': 0.29254022240638733}]

In [75]:
checkpoint_name="./img_checkpoints_500/bertimbau-base-epoch=471-val_loss=3.99.ckpt"
checkpoint = LitBertClassifier.load_from_checkpoint(checkpoint_name)
# checkpoint.bert.save_pretrained("./pictobertimbau-image-related_500e")

In [76]:
trainer.test(checkpoint,test_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1': 0.006825938355177641,
 'test_18': 0.08434909582138062,
 'test_25': 0.11506582051515579,
 'test_36': 0.14675767719745636,
 'test_9': 0.03998049721121788,
 'test_loss': 4.812594890594482,
 'test_ppl': 123.15985870361328}
--------------------------------------------------------------------------------


[{'test_ppl': 123.15985870361328,
  'test_loss': 4.812594890594482,
  'test_1': 0.006825938355177641,
  'test_9': 0.03998049721121788,
  'test_18': 0.08434909582138062,
  'test_25': 0.11506582051515579,
  'test_36': 0.14675767719745636}]

In [77]:
trainer.test(checkpoint,test_dataloaders=test_dataloader_topn)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
/tmp/ipykernel_67208/760087086.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1': 0.03998049721121788,
 'test_18': 0.2194051742553711,
 'test_25': 0.24914675951004028,
 'test_36': 0.28327643871307373,
 'test_9': 0.16821062564849854,
 'test_loss': 7.180643558502197,
 'test_ppl': 1314.706787109375}
--------------------------------------------------------------------------------


[{'test_ppl': 1314.706787109375,
  'test_loss': 7.180643558502197,
  'test_1': 0.03998049721121788,
  'test_9': 0.16821062564849854,
  'test_18': 0.2194051742553711,
  'test_25': 0.24914675951004028,
  'test_36': 0.28327643871307373}]

In [72]:
!zip  sym_checkpoints.zip checkpoints -r

	zip warning: name not matched: checkpoints

zip error: Nothing to do! (try: zip -r sym_checkpoints.zip . -i checkpoints)


In [73]:
!zip sym_logs.zip logs -r

  adding: logs/ (stored 0%)
  adding: logs/def_logs_input_mean/ (stored 0%)
  adding: logs/def_logs_input_mean/4/ (stored 0%)
  adding: logs/def_logs_input_mean/4/1e5/ (stored 0%)
  adding: logs/def_logs_input_mean/4/1e5/hparams.yaml (deflated 6%)
  adding: logs/def_logs_input_mean/4/1e5/events.out.tfevents.1671730461.fastai-1v100-6v-fin1.39200.0 (deflated 66%)
  adding: logs/def_logs_input_mean/4/1e5/events.out.tfevents.1671732355.fastai-1v100-6v-fin1.39200.1 (deflated 73%)
  adding: logs/def_logs_input_mean/4/1e5/events.out.tfevents.1671729698.fastai-1v100-6v-fin1.28717.0 (deflated 31%)
  adding: logs/def_logs_mean1/ (stored 0%)
  adding: logs/def_logs_mean1/mean1/ (stored 0%)
  adding: logs/def_logs_mean1/mean1/1e5/ (stored 0%)
  adding: logs/def_logs_mean1/mean1/1e5/events.out.tfevents.1671738708.fastai-1v100-6v-fin1.39200.5 (deflated 64%)
  adding: logs/def_logs_mean1/mean1/1e5/hparams.yaml (stored 0%)
  adding: logs/def_logs_mean1/mean1/1e5/events.out.tfevents.1671736723.fastai-1

In [74]:
from google.colab import drive
 
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!cp sym_checkpoints.zip /content/gdrive/MyDrive/Doutorado/Corpus\ construction/execution/files/sym_checkpoints.zip

In [ ]:
!cp sym_logs.zip /content/gdrive/MyDrive/Doutorado/Corpus\ construction/execution/files/sym_logs.zip